In [1]:
%load_ext autoreload
%autoreload 2

import utils
import simplenn as sn
import numpy as np

# Capa Linear

En este ejercicio debés implementar la capa `Linear`, que pesa las $I$ variables de entrada para generar $O$ valores de salida mediante la matriz de pesos $w$, de tamaño $I × O$.

Entonces, dada una entrada `x` de $N×I$ valores, donde $N$ es el tamaño de lote de ejemplos, la salida de la capa es `y=x . w`, donde $.$ es el producto matricial e `y` tiene tamaño $N×O$.


Por ejemplo, si la entrada `x` es `[[1,-1]]` (tamaño $1×2$) y la capa `Linear` tiene como parámetros `w=[[2.0, 3.0],[4.0,5.0]]` (tamaño $2×2$), entonces la salida `y` será `x . w = [ [1,-1] . [2,4], [1,-1] . [3, 5] ] = [ 1*2+ (-1)*4, 1*3+ (-1)*5] = [-2, -2] `.

Tu objetivo es implementar los métodos `forward` y `backward` de esta capa, de modo de poder utilizarla en una red neuronal.


# Creación e Inicialización

La capa `Linear` tiene un vector de parámetros `w`, que debe crearse en base a un tamaño de entrada y de salida de la capa, que debe establecerse al crearse.

Respecto a la inicialización, lo usual es hacerlo con valores aleatorios. Para ello, deberás implementar la clase `initializers.RandomNormal`, que inicializa los parámetros con una normal de media 0 y una desviación estándar que se configura al crearse.

In [101]:
# Creamos una capa Linear con 2 valores de entrada y 3 de salida
# inicializado con valores muestreados de una normal
# con media 0 y desviación estándar 1e-12

std=1e-12
linear1=sn.Linear(2,3,initializer=sn.initializers.RandomNormal(std))
print(f"Nombre de la capa: {linear1.name}")
print(f"Parámetros de la capa: {linear1.get_parameters()}")
print()

linear2 = sn.Linear(2,3,initializer=sn.initializers.RandomNormal(std))

w1 = linear1.get_parameters()["w"]
w2 = linear2.get_parameters()["w"]

print("Verificar que los pesos tengan media 0 y desviación std:")
utils.check_mean(w1,0,tol=std)
utils.check_mean(w2,0,tol=std)
utils.check_std(w1,std,tol=std)
utils.check_std(w2,std,tol=std)

print("Verificar de que las capas tienen valores distintos:")
utils.check_different(w1,w2,tol=std/10)

Nombre de la capa: Linear_196
Parámetros de la capa: {'w': array([[-1.63598503e-13,  9.87121059e-13,  8.08173120e-13],
       [ 6.77675311e-13,  6.25871941e-13,  2.41782772e-12]])}

Verificar que los pesos tengan media 0 y desviación std:
SUCCESS Mean is 8.921784412165325e-13 :) (tolerance 1e-12)
SUCCESS Mean is 3.984046799918877e-13 :) (tolerance 1e-12)
SUCCESS Std is 7.719318209290646e-13 :) (tolerance 1e-12)
SUCCESS Std is 6.264961873417658e-13 :) (tolerance 1e-12)
Verificar de que las capas tienen valores distintos:
SUCCESS Arrays are different :) (tolerance 1e-13)


# Método forward


Ahora que sabemos como crear e inicializar objetos de la capa `Linear`, comenzamos con el método `forward`, que podrás encontrar en el archivo `dense.py` de la carpeta `simplenn`.

Para verificar que la implementación de `forward` es correcta, utilizamos el inicializador `Constant`, pero luego por defecto la capa debe seguir utilizando un inicializador aleatorio como `RandomNormal`.


In [119]:
x = np.array([[3,-7],
             [-3,7]])

w = np.array([[2, 3, 4],[4,5,6]])
initializer = sn.initializers.Constant(w)

layer=sn.Linear(2,3,initializer=initializer)
y = np.array([[-22, -26, -30],
              [ 22, 26,  30]])

utils.check_same(y,layer.forward(x))

initializer = sn.initializers.Constant(-w)
layer=sn.Linear(2,3,initializer=initializer)
utils.check_same(-y,layer.forward(x))

SUCCESS Arrays are equal :) (tolerance 1e-12)
SUCCESS Arrays are equal :) (tolerance 1e-12)


# Método backward

Además del cálculo de la salida de la capa, la misma debe poder propagar hacia atrás el gradiente del error de la red. Para eso, debés implementar el método `backward` que recibe $\frac{δE}{δy}$, es decir, las derivadas parciales del error respecto a la salida (gradiente) de esta capa , y devolver $\frac{δE}{δx}$, las derivadas parciales del error respecto de las entradas de esta capa. 

## `δEδx`
Para la capa `Bias` el cálculo del gradiente respecto de la entrada `δEδx` es simple, ya que es el mismo caso que con la capa `AddConstant`. 

$ \frac{δE}{δx} =\frac{δE}{δy} $

No obstante, para esta capa también deberás implementar el gradiente con respecto a los parámetros `b`, de modo que se puedan optimizar para minimizar el error. Entonces también deberás calcular `δEδb`. Recordemos que:

$
y([x_1,x2,...,x_f])= [x_1+b_1,x_2+b_2,...,x_n+b_f]
$

Entonces, utilizando la regla de la cadena:
$
\frac{δE}{δb_i} = \frac{δE}{δy} \frac{δy_i}{δb_i} = \frac{δE}{δy_i} \frac{δy_i}{δb_i} = \frac{δE}{δy_i} \frac{δ x_i+b_i}{δb_i} = \frac{δE}{δy_i} 1 = \frac{δE}{δy_i}  
$

Es decir,

$ \frac{δE}{δx} =\frac{δE}{δy} $

## `δEδb` 

En el caso del gradiente del error con respecto a `b`, la fórmula es la misma, $ \frac{δE}{δb} =\frac{δE}{δy}$. Esto se debe a que $\frac{δy_i}{δb_i} = \frac{δ(x_i+b_i)}{δb_i} = \frac{δ(x_i+b_i)}{δx_i} =1$. 

Es decir, si vemos tanto a $b$ como a $x$ como entradas a la capa, $x+b$ es simétrico en $x$ y $b$ y por ende también lo son sus derivadas.


In [109]:
from test import check_gradient

# number of random values of x and δEδy to generate and test gradients
samples = 100
batch_size=2
features_in=3
features_out=5
input_shape=(batch_size,features_in)

# Test derivatives of a Linear layer with random values for `w`
layer=sn.Linear(features_in,features_out)
check_gradient.check_gradient_layer_random_sample(layer,input_shape,samples=samples)    


Linear_202 layer:
SUCCESS 2100 partial derivatives checked, 0 failed (tolerance 1e-07, 100 random input samples)
